In [1]:
import torchvision
from torch import nn
from pathlib import Path
import time

import torch, torchvision

import os
import pathlib
import torch
import random
import matplotlib.pyplot as plt
%matplotlib inline
from typing import Tuple, List, Dict
from torch import nn

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

from timeit import default_timer as timer 

In [10]:
class_names = ['Bacterial_spot',
                 'Curl_Virus',
                 'Early_blight',
                 'Healthy',
                 'Late_blight',
                 'Leaf_Mold',
                 'Mosaic_virus',
                 'Septoria_leaf_spot',
                 'Target_Spot',
                 'Two_spotted_spider_mite']

In [11]:
loc = Path("E:\Tomato-Classification\Data\PlantVillage")

In [12]:
model = torchvision.models.efficientnet_b2()

In [13]:
saved_model = torch.load(r"E:\Tomato-Classification\notebooks\model.pth").to('cpu')

In [14]:
data_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
device = 'cpu'

In [16]:
import gradio as gr

In [17]:
from typing import Tuple, Dict

def predict(img) -> Tuple[Dict, float]:
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()
    
    # Transform the target image and add a batch dimension
    img = data_transform(img).unsqueeze(0).to('cpu')
    
    # Put model into evaluation mode and turn on inference mode
    saved_model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(saved_model(img), dim=1)
    
    # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    # Calculate the prediction time
    pred_time = round(timer() - start_time, 5)
    
    # Return the prediction dictionary and prediction time 
    return pred_labels_and_probs, pred_time

In [18]:
import random
from PIL import Image
from pathlib import Path

# Get a list of all test image filepaths
test_data_paths = list(loc.glob("*/*.jpg"))
#image_path_list = list(loc.glob("*/*.jpg"))

# Randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

# Open the target image
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")

# Predict on the target image and print out the outputs
pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

[INFO] Predicting on image at path: E:\Tomato-Classification\Data\PlantVillage\Two_spotted_spider_mite\789cd918-1324-45be-a616-e38f9ab50a48___Com.G_SpM_FL 1189.JPG

Prediction label and probability dictionary: 
{'Bacterial_spot': 2.725256990743219e-06, 'Curl_Virus': 0.00021838702377863228, 'Early_blight': 1.146427803178085e-05, 'Healthy': 0.0004805121570825577, 'Late_blight': 2.4606538318039384e-06, 'Leaf_Mold': 0.0005337593611329794, 'Mosaic_virus': 6.200515781529248e-05, 'Septoria_leaf_spot': 0.0005692208651453257, 'Target_Spot': 0.006529205944389105, 'Two_spotted_spider_mite': 0.9915902018547058}
Prediction time: 0.06394 seconds


In [19]:
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['E:\\Tomato-Classification\\Data\\PlantVillage\\Septoria_leaf_spot\\f285de60-b746-43c9-8891-133dce784b44___Keller.St_CG 1788.JPG'],
 ['E:\\Tomato-Classification\\Data\\PlantVillage\\Early_blight\\603c6233-8cc0-4bb7-97f3-0e5db1614c63___RS_Erly.B 9398.JPG'],
 ['E:\\Tomato-Classification\\Data\\PlantVillage\\Target_Spot\\dbb50cba-a410-49af-a95f-ac53d5fc9af3___Com.G_TgS_FL 8295.JPG']]

In [20]:
import gradio as gr

# Create title, description and article strings
title = "Tomato Leaf Disease "
description = "An EfficientNetB2 feature extractor computer vision model to classify images of tomato leaf if they are healthy or infected"

# Create the Gradio demo
demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list, 
                    title=title,
                    description=description)

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://35ce184d44eaeba9aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
